In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import os
import tensorflow as tf
import random
import shutil
from shutil import copyfile
from zipfile import ZipFile
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
file_name="/content/gdrive/My Drive/Datasets/165566_377107_bundle_archive.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("Done")

Done


In [ ]:
print(len(os.listdir("/content/brain_tumor_dataset/yes")))
print(len(os.listdir("/content/brain_tumor_dataset/no")))

155
98


In [ ]:
#Creating train and test directories
try:
  os.mkdir("/content/BT/")
  os.mkdir("/content/BT/training")
  os.mkdir("/content/BT/testing")
  os.mkdir("/content/BT/training/YES")
  os.mkdir("/content/BT/training/NO")
  os.mkdir("/content/BT/testing/YES")
  os.mkdir("/content/BT/testing/NO")
except OSError:
  pass

In [ ]:
#Distrbuting dataset
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)

YES_SOURCE_DIR = "/content/brain_tumor_dataset/yes/"
TRAINING_YES_DIR = "/content/BT/training/YES/"
TESTING_YES_DIR = "/content/BT/testing/YES/"
NO_SOURCE_DIR = "/content/brain_tumor_dataset/no/"
TRAINING_NO_DIR = "/content/BT/training/NO/"
TESTING_NO_DIR = "/content/BT/testing/NO/"

split_size=.8
split_data(YES_SOURCE_DIR,TRAINING_YES_DIR,TESTING_YES_DIR,split_size)
split_data(NO_SOURCE_DIR,TRAINING_NO_DIR,TESTING_NO_DIR,split_size)


In [ ]:
#Displaying no of images in train and test folders
print(len(os.listdir("/content/BT/training/YES/")))
print(len(os.listdir("/content/BT/training/NO/")))
print(len(os.listdir("/content/BT/testing/YES/")))
print(len(os.listdir("/content/BT/testing/NO/")))

124
78
31
20


In [ ]:
#Using augmentation 
TRAINING_DIR = "/content/BT/training/"
train_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=1,
                                                    class_mode='binary',
                                                    target_size=(240, 240))

VALIDATION_DIR = "/content/BT/testing/"
validation_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest')
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=1,
                                                              class_mode='binary',
                                                              target_size=(240, 240))

Found 202 images belonging to 2 classes.
Found 51 images belonging to 2 classes.


In [ ]:
#Keras model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (2, 2), activation='relu', input_shape=(240, 240, 3)),
    tf.keras.layers.MaxPooling2D(4, 4),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(64, (2, 2), activation='relu'),
    tf.keras.layers.MaxPooling2D(4, 4),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 239, 239, 32)      416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 59, 59, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 59, 59, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 58, 58, 64)        8256      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('acc')>0.95):
                print("\nReached 95% accuracy so cancelling training!")
                self.model.stop_training = True

In [ ]:
callbacks = myCallback()
history = model.fit_generator(train_generator,
                              epochs=1000,
                              verbose=1,
                              validation_data=validation_generator,callbacks=[callbacks])

model.save('Brain_tumor_detection_model.h5')

Epoch 1/1000
241/241 [==============================] - 5s 22ms/step - loss: 0.7077 - acc: 0.6390 - val_loss: 0.6628 - val_acc: 0.7111
Epoch 2/1000
241/241 [==============================] - 5s 21ms/step - loss: 0.5886 - acc: 0.7095 - val_loss: 0.5998 - val_acc: 0.6778
Epoch 3/1000
241/241 [==============================] - 5s 21ms/step - loss: 0.6024 - acc: 0.7344 - val_loss: 0.5976 - val_acc: 0.7889
Epoch 4/1000
241/241 [==============================] - 5s 21ms/step - loss: 0.5923 - acc: 0.7303 - val_loss: 0.6391 - val_acc: 0.7444
Epoch 5/1000
241/241 [==============================] - 5s 21ms/step - loss: 0.5912 - acc: 0.7137 - val_loss: 0.5972 - val_acc: 0.7111
Epoch 6/1000
241/241 [==============================] - 5s 21ms/step - loss: 0.5854 - acc: 0.7469 - val_loss: 0.5903 - val_acc: 0.6778
Epoch 7/1000
241/241 [==============================] - 5s 21ms/step - loss: 0.5769 - acc: 0.7220 - val_loss: 0.5768 - val_acc: 0.7111
Epoch 8/1000
241/241 [==============================] -

In [ ]:
from google.colab import files
files.upload()

Saving Brain_tumor_detection_model2(95,89).h5 to Brain_tumor_detection_model2(95,89).h5


In [ ]:
from keras.models import load_model
model = load_model('Brain_tumor_detection_model2(95,89).h5')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(240, 240))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " There is a tumor")
  else:
    print(fn + " There is no tumor")

Saving no2.jpg to no2.jpg
[1.]
no2.jpg There is a tumor
